---
# 고객 등록 화장품 목록 만들기
---

In [1]:
import pymysql
import pandas as pd

In [14]:
import pandas as pd 
df = pd.read_csv('./06.customer_regis_sampling.csv')
df

,regis_num,regis_date,userID,prd_brand,prd_name,pay_date,pay_price,op_date
0,240123_F0002_00,2024-01-23,F0002,얼트루,아임쏘리포마이스킨 릴렉싱 앰플,NaN,NaN,NaN
1,240123_F0002_01,2024-01-23,F0002,닥터지,에이클리어 스팟 포 페이스 세럼,NaN,NaN,NaN
2,240123_F0002_02,2024-01-23,F0002,라운드랩,1025 독도 토너,NaN,NaN,NaN
3,240123_F0002_03,2024-01-23,F0002,아벤느,트릭세라 로션,NaN,NaN,NaN
4,240123_F0002_04,2024-01-23,F0002,에스트라,아토베리어365 하이드로 수딩크림,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
49990,240123_M9994_00,2024-01-23,M9994,메디필,히알루론산 레이어 물톡스 앰플 더블 기획,NaN,NaN,NaN
49991,240123_M9994_01,2024-01-23,M9994,닐스야드 레머디스,레머디스 와일드 로즈 뷰티 세럼,NaN,NaN,NaN
49992,240123_M9994_02,2024-01-23,M9994,닥터지,더모이스처 배리어 D 리퀴드,NaN,NaN,NaN
49993,240123_M9994_03,2024-01-23,M9994,아크네스,모이스처 플루이드,NaN,NaN,NaN


In [ ]:
df['등록번호']

In [32]:

conn = pymysql.connect(host='localhost', user='root', password='apple123!!', db = 'beaulol', charset='utf8')
cur= conn.cursor()

insert_sql = "INSERT INTO customer_regis(regis_num, regis_date, userID, prd_brand, prd_name) values(%s, %s, %s, %s, %s)"
create_sql = """ 
        create table if not exists customer_regis ( 
        regis_num VARCHAR(30) NOT NULL UNIQUE,
        regis_date DATE,
        userID VARCHAR(10) NOT NULL,
        prd_brand VARCHAR(50) NOT NULL, 
        prd_name VARCHAR(500) NOT NULL, 
        pay_date DATE, 
        pay_price VARCHAR(10),
        op_date DATE,
        PRIMARY KEY(regis_num)
        );
        """


cur.execute(create_sql)
for i in range(len(df['regis_num'])):
    cur.execute(insert_sql,(df['regis_num'][i],'2024-01-23',df['userID'][i],df['prd_brand'][i],df['prd_name'][i]))
        
conn.commit()
conn.close()

# 랜덤으로 데이터 샘플링

## DB에 저장된 고객 및 상품 정보 가져오기

In [4]:
conn = pymysql.connect(host='localhost', user='root', password='apple123!!', db = 'beaulol', charset='utf8')
cur= conn.cursor()

sql = """select * from customer_info"""
sql3 = """select * from product_info where prd_ingredients like '%,%'"""
customer_info = pd.read_sql(sql, conn)
product_info = pd.read_sql(sql3,conn)

/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_988/688440226.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customer_info = pd.read_sql(sql, conn)
/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_988/688440226.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product_info = pd.read_sql(sql3,conn)


In [5]:
customer_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   userID  9999 non-null   object
 1   성함      9999 non-null   object
 2   주소      9999 non-null   object
 3   이메일     9999 non-null   object
 4   연락처     9999 non-null   object
 5   아이디     9999 non-null   object
 6   비밀번호    9999 non-null   object
 7   생년월일    9999 non-null   object
 8   성별      9999 non-null   object
dtypes: object(9)
memory usage: 703.2+ KB


In [6]:
product_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393 entries, 0 to 392
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   prd_name         393 non-null    object
 1   prd_brand        393 non-null    object
 2   prd_price        393 non-null    object
 3   prd_capa         393 non-null    object
 4   prd_ingredients  393 non-null    object
 5   prd_exp          393 non-null    object
 6   prd_category     393 non-null    object
dtypes: object(7)
memory usage: 21.6+ KB


## 랜덤뽑기 잘 되는지 확인

In [7]:
import numpy as np
product = np.random.choice(list(product_info.loc[product_info['prd_category']=='세럼']['prd_name'].unique()),1)[0]
product

'레티놀 흔적 리페어 앰플'

In [8]:
product_info.loc[product_info['prd_name']==product]['prd_brand']

89    시드물
Name: prd_brand, dtype: object

## DB 저장

In [9]:
conn = pymysql.connect(host='localhost', user='root', password='apple123!!', db = 'beaulol', charset='utf8')
cur= conn.cursor()

create_sql = """ 
        create table if not exists customer_regis ( 
        regis_num VARCHAR(30) NOT NULL UNIQUE,
        regis_date DATE,
        userID VARCHAR(10) NOT NULL,
        prd_brand VARCHAR(50) NOT NULL, 
        prd_name VARCHAR(500) NOT NULL, 
        pay_date DATE, 
        pay_price VARCHAR(10),
        op_date DATE,
        PRIMARY KEY(regis_num)
        );
        """

cur.execute(create_sql)
        
conn.commit()
conn.close()

In [10]:
conn = pymysql.connect(host='localhost', user='root', password='apple123!!', db = 'beaulol', charset='utf8')
cur= conn.cursor()

insert_sql = "INSERT INTO customer_regis(regis_num, regis_date ,userID, prd_brand, prd_name) values(%s, %s, %s, %s, %s)"

for user in customer_info['userID']:
    for i in range(0,5):
        if i == 0 or i == 1:
            product = np.random.choice(list(product_info.loc[product_info['prd_category']=='세럼']['prd_name'].unique()),1)[0]
        if i == 2:
            product = np.random.choice(list(product_info.loc[product_info['prd_category']=='스킨토너']['prd_name'].unique()),1)[0]
        if i == 3:
            product = np.random.choice(list(product_info.loc[product_info['prd_category']=='로션']['prd_name'].unique()),1)[0]
        if i == 4:
            product = np.random.choice(list(product_info.loc[product_info['prd_category']=='크림']['prd_name'].unique()),1)[0]
        brand = product_info.loc[product_info['prd_name']==product]['prd_brand'].iloc[0]
        reginum = '240123' + "_" + str(user) + "_0" + str(i)
        
        cur.execute(insert_sql,(reginum,'2024-01-23',user,brand,product))
        
conn.commit()
conn.close()

# DB에 저장한 데이터 데이터프레임으로 파일 저장

In [11]:
conn = pymysql.connect(host='localhost', user='root', password='apple123!!', db = 'beaulol', charset='utf8')
cur= conn.cursor()

from_sql = "select * from customer_regis"
dfs = pd.read_sql(from_sql,conn)
dfs

/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_988/1284692759.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfs = pd.read_sql(from_sql,conn)


,regis_num,regis_date,userID,prd_brand,prd_name,pay_date,pay_price,op_date
0,240123_F0002_00,2024-01-23,F0002,얼트루,아임쏘리포마이스킨 릴렉싱 앰플,None,None,None
1,240123_F0002_01,2024-01-23,F0002,닥터지,에이클리어 스팟 포 페이스 세럼,None,None,None
2,240123_F0002_02,2024-01-23,F0002,라운드랩,1025 독도 토너,None,None,None
3,240123_F0002_03,2024-01-23,F0002,아벤느,트릭세라 로션,None,None,None
4,240123_F0002_04,2024-01-23,F0002,에스트라,아토베리어365 하이드로 수딩크림,None,None,None
...,...,...,...,...,...,...,...,...
49990,240123_M9994_00,2024-01-23,M9994,메디필,히알루론산 레이어 물톡스 앰플 더블 기획,None,None,None
49991,240123_M9994_01,2024-01-23,M9994,닐스야드 레머디스,레머디스 와일드 로즈 뷰티 세럼,None,None,None
49992,240123_M9994_02,2024-01-23,M9994,닥터지,더모이스처 배리어 D 리퀴드,None,None,None
49993,240123_M9994_03,2024-01-23,M9994,아크네스,모이스처 플루이드,None,None,None


In [13]:
dfs.to_csv('./06.customer_regis_sampling.csv',index=False)